# Scrabu Prototype (for a Single Shipment Number)

This notebook is for creating a prototype of the Scrabu project. The goal is to download DHL pages for a specific shipment number and scrap the shipment information from it.

In [165]:
# request & response handler
import requests
import numpy as np

def generate_shipment_numbers(shipment_number = 340434188193324407):
    shipment_number = 340434188193324407
    shipment_numbers_list = []
    for i in range(1, 100):
        shipment_numbers = list(map(int, str(shipment_number)))
        shipment_number = str(shipment_number).rjust(20, '0')
        shipment_numbers_list.append(shipment_number)
    return shipment_numbers_list


def request(shipment_numbers_list, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number=None):
    
    for shipment_number in shipment_numbers_list:
        request_url = start_url + shipment_number
        return requests.get(request_url).content


In [166]:
import requests

def response(request, shipment_number):
    for shipment_number in shipment_numbers_list:
        html_response = request(shipment_number)
        return requests.get(html_response).content

In [167]:
# parser
from lxml import etree
import json

def html_to_json(html, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number=None):
    html_tree = etree.HTML(html)
    
    def clean_json(json):
        start = json.find('JSON.parse(')
        end = json.find('"),', start)
        json = json[start:end]
        json = json.replace('JSON.parse("', '')
        json = json.replace('\\', '')
        return json
    
    def find_json_element(html_tree):
        json_element = html_tree.xpath('//div')
        return str(etree.tostring(json_element[0]))
    
    dirty_json = find_json_element(html_tree)
    json_string = clean_json(dirty_json)
    return json.loads(json_string)

In [168]:
shipment_details_json = html_to_json(html_response)

In [169]:
import datetime


def shipment_details(shipment_details_json, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number="00340434188193324407"):
    delivery_history_dict = {}
    
    delivery_history_dict["shipment_number"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsnummern"].get("sendungsnummer")
    delivery_history_dict["crawltime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    delivery_history_dict["url"] = start_url + shipment_number 
    delivery_history_dict["events"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsverlauf"].get("events", [])
    return delivery_history_dict

In [171]:
shipment_history = shipment_details(shipment_details_json)
print(shipment_history)

{'shipment_number': '00340434188193324407', 'crawltime': '2019-10-21 14:34:18', 'url': 'https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193324407', 'events': [{'datum': '2019-10-07T16:27:00+02:00', 'status': 'Die Sendung wurde elektronisch angeku00FCndigt. Sobald die Sendung von uns bearbeitet wurde, erhalten Sie weitere Informationen.'}, {'datum': '2019-10-08T17:12:00+02:00', 'ort': 'Bremen', 'status': 'Die Sendung wurde von DHL bearbeitet und wird fu00FCr den Weitertransport in die Region des Empfu00E4ngers vorbereitet.'}, {'datum': '2019-10-09T10:25:00+02:00', 'ort': 'Neuwied', 'status': 'Die Sendung ist in der Region des Empfu00E4ngers angekommen und wird im nu00E4chsten Schritt zur Zustellbasis transportiert.'}, {'datum': '2019-10-10T05:58:00+02:00', 'ort': 'Bonn MechZB', 'status': 'Die Sendung wird fu00FCr die Verladung ins Zustellfahrzeug vorbereitet.'}, {'datum': '2019-10-10T09:19:00+02:00', 'status': 'Die Sendung wurde in das Zustellf

In [191]:
import json
def save_dictionary(shipment_history=None, filename=None):
    with open(filename, 'w') as f:
        json.dump(shipment_history, f)    

In [194]:
for shipment_number in shipment_history:
    shipment_number = shipment_history.get(shipment_number)
    filename = "../data/"+shipment_number+".json"
    save_dictionary(shipment_history=shipment_history, filename=filename)

FileNotFoundError: [Errno 2] No such file or directory: '../data/https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=00340434188193324407.json'